### Financial Text Sentiment Analysis


In [35]:
import os
from dotenv import load_dotenv
from newsapi import NewsApiClient
from transformers import pipeline
from datetime import datetime

load_dotenv()
key = os.getenv("NEWSAPI_KEY")

newsapi = NewsApiClient(api_key=key)
pipe = pipeline("text-classification", model="ProsusAI/finbert")


today = datetime.today().strftime('%Y-%m-%d')

keyword = 'palantir'
news = newsapi.get_everything(q=keyword,
                              from_param='2025-06-02',
                              to=today,
                              language='en',
                              sort_by='relevancy',
                                )

articles = news['articles']
articles = [article for article in articles if keyword.lower() in article['title'].lower() or keyword.lower() in article['description'].lower()]

total_weight = 0 
weighted_sum = 0
label_map = {"positive":1, "neutral":0, "negative":-1}

for i, article in enumerate(articles):
    
    print(f"Title: {article['title']}")
    print(f"Link: {article['url']}")
    print(f"Description: {article['description']}")
    
    sentiment = pipe(article['content'])[0]
    score = label_map.get(sentiment['label'].lower(), 0)
    weight = sentiment['score']
    
    print(f"Sentiment {sentiment['label']}, Score: {sentiment['score']}")
    print('-' * 40) 
    
    weighted_sum += score * weight
    total_weight += weight


final_score = weighted_sum / total_weight if total_weight > 0 else 0
print(f"Overall Sentiment: {'Positive' if final_score > 0.2 else 'Negative' if final_score < -0.2 else 'Neutral'} {final_score}")
    



Device set to use cpu


Title: Is MAGA Turning on Trump’s Buddy Palantir?
Link: https://gizmodo.com/is-maga-turning-on-trumps-buddy-palantir-2000610499
Description: Rightwing America doesn't seem too pleased that the defense contractor is playing a pivotal role in the Trump government.
Sentiment negative, Score: 0.9166317582130432
----------------------------------------
Title: The Palantir job that grows startup founders
Link: https://www.businessinsider.com/palantirs-forward-deployed-engineer-role-churns-out-startup-founders-2025-6
Description: Forward Deployed Software Engineers at Palantir gain startup-ready skills through direct client work. Many have left the firm to become founders.
Sentiment neutral, Score: 0.8605000376701355
----------------------------------------
Title: YC partners say AI founders are closing huge deals fast by taking a page out of Palantir's early playbook
Link: https://www.businessinsider.com/ai-founders-close-big-deals-palantir-forward-deployed-engineers-yc-2025-6
Description: A